In [1]:
import pandas as pd
import pipeline as p
import datetime as dt

%load_ext autoreload
%autoreload 2

In [21]:
stats_df = p.open_pkl('Data/stats_df.pkl')

In [3]:
elo_df = p.open_pkl('Data/elo_df.pkl')

In [4]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
0,2007-09-06,2007,0,NaN,IND,NO,1653.923,1515.973,0.762833,0.237167,1668.974,1500.922,41.0,10.0
1,2007-09-09,2007,0,NaN,DAL,NYG,1490.892,1494.060,0.588056,0.411944,1510.108,1474.844,45.0,35.0
2,2007-09-09,2007,0,NaN,LAR,CAR,1478.632,1511.942,0.545480,0.454520,1448.656,1541.917,13.0,27.0
3,2007-09-09,2007,0,NaN,LAC,CHI,1646.204,1574.744,0.686870,0.313130,1660.857,1560.090,14.0,3.0
4,2007-09-09,2007,0,NaN,WSH,MIA,1448.841,1489.941,0.534341,0.465659,1461.613,1477.169,16.0,13.0


Compute Deltas of Elo1 and Point Diff (easier now that pre and post are provided)

In [5]:
elo_df['elo_delta'] = elo_df['elo1_post'] - elo_df['elo1_pre']
elo_df['point_diff'] = elo_df['score1'] - elo_df['score2']

In [6]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2,elo_delta,point_diff
0,2007-09-06,2007,0,NaN,IND,NO,1653.923,1515.973,0.762833,0.237167,1668.974,1500.922,41.0,10.0,15.051,31.0
1,2007-09-09,2007,0,NaN,DAL,NYG,1490.892,1494.060,0.588056,0.411944,1510.108,1474.844,45.0,35.0,19.216,10.0
2,2007-09-09,2007,0,NaN,LAR,CAR,1478.632,1511.942,0.545480,0.454520,1448.656,1541.917,13.0,27.0,-29.976,-14.0
3,2007-09-09,2007,0,NaN,LAC,CHI,1646.204,1574.744,0.686870,0.313130,1660.857,1560.090,14.0,3.0,14.653,11.0
4,2007-09-09,2007,0,NaN,WSH,MIA,1448.841,1489.941,0.534341,0.465659,1461.613,1477.169,16.0,13.0,12.772,3.0


In [7]:
len(elo_df)

3204

In [8]:
len(stats_df)

6386

In [58]:
df2 = pd.merge(elo_df, stats_df, right_on=['Team','Date'], left_on=['team1','date'], copy=False)
df2 = pd.merge(df2, stats_df, right_on=['Team','Date'], left_on=['team2','date'], copy=False, suffixes=('1','2'))

In [13]:
len(df2)

3193

In [59]:
cols_to_drop = [
 'Date2',
 'Day2',
 'Games1',
 'Games2',
 'Location2',
 'OT2',
 'Opp1',
 'Opp2',
 'Record1',
 'Record2',
 'Result1',
 'Result2',
 'Season2',
 'Time2',
 'Week2',
 'Win%1',
 'Win%2',
 'Wins1',
 'Wins2',
 'date',
 'elo1_post',
 'elo2_post',
 'score1',
 'score2',
    'team1', 
    'team2'
]

In [60]:
df2.drop(columns=cols_to_drop, inplace=True)

In [61]:
len(df2.columns)

68

In [62]:
cols_to_rename = [
 'Date1',
 'Day1',
 'Location1',
 'OT1',
 'Opp1',
 'Season1',
 'Time1',
 'Week1']

rename_dict = {x: x[:-1] for x in cols_to_rename}
df2.rename(columns=rename_dict, inplace=True)

In [63]:
p.pkl_this('Data/df_for_model.pkl', df2)